In [19]:
# simsiam
!experiment_name="debug";cd ../ ;\
    name_date="${experiment_name}/$(date +%Y-%m%d-%H%M%S)";\
    dir="./results/${name_date}";mkdir -p ${dir};dir_log="${dir}/output.log";\
CUDA_VISIBLE_DEVICES=6,7 torchrun --nproc_per_node=2 --master_port=1234 train_multi_GPU.py \
    --wandb True --wandb_model dryrun --sync_bn False --amp True --aux False \
    --model_name deeplabv3_resnet50 --pre_trained deeplabv3_resnet50_coco.pth \
    --weight_only_backbone True \
    --data_path pascal-voc-2012 --num_classes 21 \
    --epochs 100 --batch_size 6 --batch_size_val 4 --memory_size 0 \
    --contrast -1 --loss_name simsiam --L1_loss 0.1 --L2_loss 0.1 --L3_loss 0.1\
    --name_date $name_date \
    2>&1 | tee $dir_log

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 0): env://
| distributed init (rank 2): env://
| distributed init (rank 3): env://
debug/2023-0322-124413
Namespace(data_path='cityscapes', device='cuda', num_classes=19, batch_size=6, batch_size_val=4, aux=False, start_epoch=0, epochs=100, sync_bn=False, workers=4, lr=0.0001, momentum=0.9, weight_decay=0.0001, print_freq=10, checkpoint_dir='./results/debug/2023-0322-124413', resume='', test_only=False, world_size=4, dist_url='env://', amp=True, seed=304, name_date='debug/2023-0322-124413', wandb=False, wandb_model='dryrun', model_name='deeplabv3_resnet50', project_dim=128, loss_name='simsiam', contrast=-1, pre_trained='resnet50-imagenet